# Fitting a distribution to radius to maximum wind observations

The historical record has only sparse observations of radius to maximum winds $R_{max}$ in the Australian region (2002 onwards). As in Vickery _et al._ (2000), we assume that $R_{max}$ fits a log-normal distribution. Powell _et al._ (2005) provide a functional form for the distribution (their Eq. 7), and we will use this as a first estimate. The resulting model is intended for application in setting $R_{max}$ values for stochastically generated storms in TCRM.

Note that this model describes the log normal distribution of $R_{max}$ in kilometres -- Powell _et al._ define their model in nautical miles.

In [1]:
%matplotlib inline

from __future__ import division, print_function
import os
from os.path import join as pjoin
from matplotlib import pyplot as plt
from datetime import datetime, timedelta

from Utilities.metutils import convert

import numpy as np
import scipy.stats as stats

import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.api as sm
import statsmodels.nonparametric.api as smnp
from six import string_types
    
from statsmodels.tools.tools import ECDF

from lmfit import Model, Minimizer, fit_report, conf_interval, printfuncs, report_fit
import corner

import seaborn as sns
from seaborn.utils import _kde_support
sns.set_style("darkgrid")
sns.set_context("poster")

First a short function to convert the formatted latitude/longitude values to actual numbers.

In [2]:
def convertLatLon(strval):
    """
    Convert a string representing lat/lon values from '140S to -14.0, etc.
    
    :param str strval: string containing the latitude or longitude.
    
    :returns: Latitude/longitude as a float value.
    
    """
    hemi = strval[-1].upper()
    fval = float(strval[:-1]) / 10.
    if (hemi == 'S') | (hemi == 'W'): 
        fval *= -1
    if (hemi == 'E') | (hemi == 'W'):
        fval = fval % 360
    return fval
            

Define the data structure and a small function to load a file. This uses the JTWC data format, described [here](http://www.usno.navy.mil/NOOC/nmfc-ph/RSS/jtwc/best_tracks/shindex.php). 

In [3]:
COLNAMES = ['BASIN','Number', 'Datetime','TECHNUM', 'TECH','TAU', 'Latitude', 'Longitude', 'Windspeed','Pressure',
            'Status', 'RAD', 'WINDCODE','RAD1', 'RAD2','RAD3', 'RAD4','Poci', 'Roci','rMax', 'GUSTS','EYE',
            'SUBREGION','MAXSEAS', 'INITIALS','DIR', 'SPEED','STORMNAME', 'DEPTH','SEAS',
            'SEASCODE','SEAS1', 'SEAS2','SEAS3', 'SEAS4'] 

COLTYPES = ['|S2', 'i', datetime, 'i', '|S4', 'i', 'f', 'f', 'f', 'f', 
            '|S4', 'f', '|S3', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f', 'f',
            '|S1', 'f', '|S3', 'f', 'f', '|S10', '|S1', 'f', 
            '|S3', 'f', 'f', 'f', 'f']
COLUNITS = ['', '', '', '', '', '', '', '', 'kts', 'hPa', 
            '', 'nm', '', 'nm', 'nm', 'nm', 'nm', 'hPa', 'nm', 'nm', 'kts', 'nm',
            '', '', '', 'degrees', 'kts', '', '', '',
            '', '', '', '', '']
DATEFORMAT = "%Y%m%d%H"
dtype = np.dtype({'names':COLNAMES, 'formats':COLTYPES})
converters = {
    1: lambda s: s.strip(' ,'),
    2: lambda s: datetime.strptime(s.strip(' ,'), DATEFORMAT),
    6: lambda s: float(convertLatLon(s.strip(' ,'))),
    7: lambda s: float(convertLatLon(s.strip(' ,'))),
    8: lambda s: s.strip(' ,'),
    9: lambda s: s.strip(' ,'),
    10: lambda s: s.strip(' ,'),
    11: lambda s: convert(float(s.strip(' ,') or 0), COLUNITS[11], 'km'),
    12: lambda s: s.strip(' ,'),
    13: lambda s: convert(float(s.strip(' ,') or 0), COLUNITS[13], 'km'),
    14: lambda s: convert(float(s.strip(' ,') or 0), COLUNITS[14], 'km'),
    15: lambda s: convert(float(s.strip(' ,') or 0), COLUNITS[15], 'km'),
    16: lambda s: convert(float(s.strip(' ,') or 0), COLUNITS[16], 'km'),
    17: lambda s: float(s.strip(' ,')),
    18: lambda s: convert(float(s.strip(' ,') or 0), COLUNITS[18], 'km'),
#    19: lambda s: float(s.strip(' ,'))
    19: lambda s: convert(float(s.strip(' ,') or 0), COLUNITS[19], 'km')
}
delimiter = (3,4,12,4,6,5,7,7,5,6,4,5,5,6,6,6,6,6,6,5,5,5,5)
skip_header = 0
usecols = tuple(range(23))
missing_value = ""
filling_values = 0

def loadData(filename):
    try:
        data = np.genfromtxt(filename, dtype, delimiter=delimiter, skip_header=skip_header, 
                             converters=converters, missing_values=missing_value, 
                             filling_values=filling_values, usecols=usecols, autostrip=True, invalid_raise=False)
    except IndexError:
        try:
            data = np.genfromtxt(filename, dtype, delimiter=delimiter, skip_header=skip_header, 
                             converters=converters, missing_values=missing_value, 
                             filling_values=filling_values, usecols=tuple(range(18)), autostrip=True, invalid_raise=False)
        except IndexError:
            data = np.genfromtxt(filename, dtype, delimiter=[3,4,12,4,6,5,7,7,5], skip_header=skip_header, 
                             converters=converters, missing_values=missing_value, 
                             filling_values=filling_values, usecols=tuple(range(9)), autostrip=True, invalid_raise=False)
    return data


Often the b-deck files contain multiple records with the same time stamp. This is to record information on different wind speed radii (e.g. the radius to 34-knot winds, 48-knot winds, etc.). We can quickly filter out this extra information using [`numpy.unique()`](http://docs.scipy.org/doc/numpy/reference/generated/numpy.unique.html). Additional filtering restricts to a known domain and only those storms that are of Tropical Storm or Typhoon strength, those that have radius to maximum wind records, and those that include a pressure for the outermost closed isobar ($p_{oci}$).

In [4]:
def filterData(data):
    datetimes, idx = np.unique(data['Datetime'], True)
    filter1 = (data['Status'][idx] == 'TS') | (data['Status'][idx] == 'TY')
    filter2 = (data['Longitude'][idx] >= 60.) & (data['Longitude'][idx] <= 180.)
    filter3 = (data['rMax'][idx] >= 0.1)
    filter4 = (data['Poci'][idx] > 0.1)
    subsidx = np.nonzero(filter1 & filter2 & filter3 & filter4)
    return data[subsidx]

Now churn through the best-track files (unmodified) and pull out $R_{max}$, $p_c$, $p_{oci}$ and latitude values. This assumes you have the JTWC best track files somewhere locally - no download performed.

In [5]:
def processfiles(path, basin):
    rmax = np.array([])
    prs = np.array([])
    lat = np.array([])
    poci = np.array([])
    for root, dirs, files in os.walk(path):
        if root.endswith(basin):
            for file in files:
                data = loadData(pjoin(root, file))
                if 'Status' in data.dtype.names:
                    data = filterData(data)
                    if 'rMax' in data.dtype.names:
                        rmax = np.append(rmax, data['rMax'])
                        prs = np.append(prs, data['Pressure'])
                        poci = np.append(poci, data['Poci'])
                        lat = np.append(lat, data['Latitude'])
    return rmax, prs, poci, lat

In [6]:
inputPath = "C:\\WorkSpace\\data\\Raw\\best_tracks"
rmax, prs, poci, lat = processfiles(inputPath, 'sh')
#outputFile = pjoin(inputPath, "rmax-sh.csv")
#np.savetxt(outputFile, np.column_stack((rmax, prs, poci, lat)), delimiter=',', fmt='%6.1f')

Now we test the first hypothesis - that the distribution of $R_{max}$ is represented by a log-normal distribution. Plot up the probability distribution function, with a kernel density estimate and a fitted log-normal distribution.

In [7]:
print("Parameter estimates:       Shape; Location (fixed);    Scale;    Mean")
fig, ax = plt.subplots(1,1)
sns.distplot(rmax, bins=np.arange(0, 101, 10),
             kde_kws={'clip':(0, 100), 'label':"KDE"}, ax=ax)

shape, loc, scale = stats.lognorm.fit(rmax, scale=np.mean(rmax), floc=0)
print("Southern hemisphere basin: ", shape, loc, scale, np.mean(rmax))
x = np.arange(1, 201)
v = stats.lognorm.pdf(x, shape, loc=loc, scale=scale)
fcdf = stats.lognorm.cdf(np.sort(rmax), shape, loc=loc, scale=scale)

ax.plot(x, v, label="Lognormal fit")
ax.legend(loc=0)
ax.set_xlabel(r'$R_{max}$ (km)')
ax.set_ylabel('Probability')
ax.set_xlim((0, 100))
ax.set_title("Southern hemisphere (2002-2014)")


fig.tight_layout()
sns.despine()

Compare the empirical CDF to the fitted CDF.

In [8]:
ecdf = ECDF(rmax, side='left')

plt.plot(np.sort(rmax), ecdf.y[1:])
plt.plot(np.sort(rmax), fcdf, 'r' )
rsq = stats.pearsonr(np.sort(rmax), fcdf)[0]**2
plt.text( 10, 0.9, r"$R^{2}$ = %f"%rsq)

## Fitting to multiple parameters

In this approach, the natural logarithm of $R_{max}$ is modelled as follows:

$\ln R_{max} = \alpha + \beta \Delta p + \gamma \exp^{(-\delta \Delta p^2)} + \zeta |\lambda| + \varepsilon$

The constants are determined by a generalised linear model, $\Delta p$ is the central pressure deficit (hPa), $\lambda$ the latitude and $\varepsilon$ is a normal random variable with zero mean. We choose an exponential decay function to ensure physically realistic behaviour at large $\Delta p$. Other models examined were quadratic in $\Delta p$, which produced unrealistic behaviour when $\Delta p$ increased beyond 100 hPa.

Additional filtering is needed here to remove records where the pressure of the outermost closed isobar ($p_{oci}$) is not known.

In [9]:
def filterPoci(field, poci):
    filter1 = (poci >= 0.1)
    subsidx = np.nonzero(filter1)
    return field[subsidx]

rmax = filterPoci(rmax, poci)
dp = filterPoci(poci, poci) - filterPoci(prs, poci)
dp = np.extract(np.nonzero(rmax), dp)
dpsq = dp*dp
expdp = np.exp(-dp)
expdpsq = np.exp(-dpsq)
lat = filterPoci(lat, poci)
lat = np.extract(np.nonzero(rmax), lat)
rmax = np.extract(np.nonzero(rmax), rmax)

latsq = lat*lat

Now fit a model, based on the functional form given above. Start by defining the independent variables ($\Delta p$, $\lambda$) and dependent variable (in this case $\ln(R_{max})$). This is followed by defining individual components of the hypothesised model, namely a linear component in $\Delta p$, a linear component in $|\lambda|$ and an exponential decay term for $\Delta p^2$.

In [10]:
X = np.column_stack((dp, lat))
y = np.log(rmax)

In [11]:
def exp_dpsq(x, gamma, delta):
    dp = x[:,0]
    return gamma*np.exp(-delta*dp*dp)

def lin_dp(x, alpha, beta):
    dp = x[:,0]
    return alpha + beta*dp

def lin_lat(x, zeta):
    lat = np.abs(x[:,1])
    return zeta*lat

Construct the model using the `lmfit` `Model` objects, define the model parameters (i.e. the coefficients), and build a fuction that returns the residual of the model fit for a given solution, which will be used in a minimizer fuction.

In [12]:
rmod = Model(lin_dp) + Model(exp_dpsq) + Model(lin_lat)
params = rmod.make_params(alpha=1., beta=-0.001, gamma=.1, delta=.001, zeta=.001)
def resid(p):
    return p['alpha'] + p['beta']*X[:,0] + p['gamma']*np.exp(-p['delta']*X[:,0]*X[:,0]) + p['zeta']*np.abs(X[:,1]) - y

mini = Minimizer(resid, params)
result = mini.minimize()
print(fit_report(result.params))
ci = conf_interval(mini, result)
printfuncs.report_ci(ci)

We can examine the range of solutions that are supported by the observations, using Markov Chain Monte Carlo sampling of the posterior probability distribution. We can use this to obtain the credible intervals for the parameters (the Bayesian equivalent of confidence intervals).

In [13]:
rr = mini.emcee(burn=500)

In [14]:
ll = [r'$\{0}$'.format(v) for v in rr.var_names]
with sns.plotting_context("notebook"):
    corner.corner(rr.flatchain, labels=ll, truths=list(rr.params.valuesdict().values()),
              no_fill_contours=True, fill_contours=False, plot_density=False,
              quantiles=[0.05, 0.5, 0.95],
                 data_kwargs=dict(color='r', alpha=0.01),
             contour_kwargs=dict(color='g'))

In [15]:
print(report_fit(rr.params))

In [17]:
print(rr.params)

In [18]:
print(r'alpha = {0}'.format(rr.params['alpha'].value))
print(r'beta = {0}'.format(rr.params['beta'].value))
print(r'gamma = {0}'.format(rr.params['gamma'].value))
print(r'delta = {0}'.format(rr.params['delta'].value))
print(r'zeta = {0}'.format(rr.params['zeta'].value))

In [19]:
result = rmod.fit(y, x=X, params=params)
print(result.fit_report())


In [21]:
result.params = rr.params

In [22]:
plt.plot(X[:,0], y,         'bo')
#plt.plot(X[:,0], result.init_fit, 'k--')
plt.plot(X[:,0], result.best_fit, 'r.')
plt.show()

We can inspect the individual model components of the model and their contribution to the solution. Using the `eval_components` method from the `result` object allows us to inspect each model component separately. If one wanted, you could plot them to visualise the relationships.

In [23]:
comps = result.eval_components()
print(comps)

Now examine the residuals to formulate a method of describing the random innovations required in a stochastic modelling framework. We assume the residuals are normally distributed and test for normality to confirm our hypothesis.

In [24]:
fig, (ax0, ax1) = plt.subplots(1, 2)

ax = sns.distplot(result.residual, kde_kws={'label':'Residuals', 'linestyle':'--'}, ax=ax0, norm_hist=True)
pp = sm.ProbPlot(result.residual, stats.norm, fit=True)
pp.qqplot('Normal', 'Residuals', line='45', ax=ax1, color='gray',alpha=0.5)
fig.tight_layout()
x = np.linspace(-2, 2, 1000)

ax0.legend(loc=0)

fp = stats.norm.fit(result.residual)
ax0.plot(x, stats.norm.pdf(x, fp[0], fp[1]), label='Normal', color='r')
print(fp)
print(stats.mstats.normaltest(result.residual))
ax0.legend()

The p-value indicates the normal distribution is a good description of the residuals of the model. The quantile-quantile plot (right) shows the model residuals are slightly fat-tailed for higher values, but are close to normal within 2 standard deviations of the mean. We can then use normally distributed values, with variancce of 0.33, to inform our stochastic model.

### Functional form
The basic functional form of the model can readily be plotted, using the `eval` method on the `result` object. All that is needed are input values representing the pressure deficit and latitude values we want to explore. Here, we calculate the mean $R_{max}$ value for $\Delta p$ values between 0 and 100 hPa, and latitude ranging from 30$^{\circ}$S to 2$^{\circ}$S. This allows us to demonstrate the influence of $\Delta p$ and $\lambda$ on $R_{max}$, without the random variations.

In [26]:
deltap = np.linspace(0, 100, 100)
lats = np.arange(-30, -1, 4)
#lats = np.arange(2, 31, 4)
fig, ax = plt.subplots(1,1)
sns.set_palette("RdBu", 10)
for l in lats:
    xx = np.column_stack((deltap, l*np.ones(len(deltap))))
    yy = result.eval(x=xx)
    ax.plot(deltap, np.exp(yy), label="%d"%l)
ax.set_ylabel(r"$R_{max}$ (km)")
ax.set_xlabel(r"$\Delta p$ (hPa)")
ax.legend(loc=1)

In the mean, $R_{max}$ starts out around 80 km for weak storms and rapidly decreases as intensity increases to $\Delta p$ = 40 hPa. At this point, $R_{max}$ is close to 30 km, and then steadily reduces for increasing intensity. Lower latitude storms are, in the mean, smaller than at high latitudes. 

### Complete model
We construct the complete model by combining the functional form with a random innovation which is based on the residuals from the fitted model. In this case, the random innovations are sampled from a normal distribution with zero mean and $\sigma$ = 0.33.

Using observed values of $\Delta p$ and $\lambda$, we can then generate a plot of observed and simulated $R_{max}$ values. 

In [27]:
nx = len(dp)
ind = np.random.choice(np.arange(nx), nx, replace=True)
dp0 = dp[ind]
l0 = lat[ind]

xx = np.column_stack((dp0, l0))
yy = result.eval(x=xx) + np.random.normal(scale=0.33, size=nx)


rm = np.exp(yy)
fig, ax = plt.subplots(1, 2, sharey=True)
ax[0].scatter(dp0, rm, c=np.abs(l0), cmap=sns.light_palette('blue', as_cmap=True), s=40, label='Model', alpha=0.5)
ax[0].scatter(dp, rmax, c='w', edgecolor='r', s=50, marker='+', label='Observations')
ax[0].set_xlim(0, 100)
ax[0].set_xlabel(r"$\Delta p$ (hPa)")
ax[0].set_ylabel(r"$R_{max}$ (km)")
ax[0].set_ylim(0, 200)
ax[0].legend(loc=1)
ax[0].grid(True)

ax[1].scatter(l0, rm, c=np.abs(l0), cmap=sns.light_palette('blue', as_cmap=True), s=40, label='Model', alpha=0.5)
ax[1].scatter(lat, rmax, c='w', edgecolor='r', s=50, marker='+', label='Observations')
ax[1].set_xlim(-30, 0)
ax[1].set_xlabel(r"Latitude")
ax[1].set_ylim(0, 200)
ax[1].legend(loc=1)
ax[1].grid(True)

fig.tight_layout()

In [28]:
def bivariate_kde(x, y, bw='scott', gridsize=100, cut=3, clip=None):
    if isinstance(bw, string_types):
        bw_func = getattr(smnp.bandwidths, "bw_" + bw)
        x_bw = bw_func(x)
        y_bw = bw_func(y)
        bw = [x_bw, y_bw]
    elif np.isscalar(bw):
        bw = [bw, bw]

    if isinstance(x, pd.Series):
        x = x.values
    if isinstance(y, pd.Series):
        y = y.values

    kde = smnp.KDEMultivariate([x, y], "cc", bw)
    x_support = _kde_support(x, kde.bw[0], gridsize, cut, [x.min(), x.max()])# clip[0])
    y_support = _kde_support(y, kde.bw[1], gridsize, cut, [y.min(), y.max()])#clip[1])
    xx, yy = np.meshgrid(x_support, y_support)
    z = kde.pdf([xx.ravel(), yy.ravel()]).reshape(xx.shape)
    return xx, yy, z

def l2score(obs, model):
    return np.linalg.norm(obs - model, np.inf)

xx, yy, odp_rmax = bivariate_kde(dp,  rmax, bw='scott')
xx, yy, mdp_rmax = bivariate_kde(dp0, rm, bw='scott')

xx, yy, olat_rmax = bivariate_kde(lat,  rmax, bw='scott')
xx, yy, mlat_rmax = bivariate_kde(l0, rm, bw='scott')


l2rmdp = l2score(odp_rmax, mdp_rmax)
l2rmlat = l2score(olat_rmax, mlat_rmax)


fig, ax = plt.subplots(1, 1)
levs = np.arange(0.01, 0.11, 0.01)
ax = sns.kdeplot(dp, rmax, cmap='Reds', kwargs={'levels':levs}, shade=True, shade_lowest=False)
ax = sns.kdeplot(dp0, rm, cmap='Blues', kwargs={'levels':levs})
ax.set_xlim(0, 100)
ax.set_xlabel(r"$\Delta p$ (hPa)")
ax.set_ylabel(r"$R_{max}$ (nm)")
ax.set_ylim(0, 100)
ax.grid(True)

red = sns.color_palette("Reds")[-2]
blue = sns.color_palette("Blues")[-2]
ax.text(80, 90, "Observed", color=red)
ax.text(80, 80, "Model", color=blue)
ax.text(80, 70, r"$l_2=${0:.3f}".format(l2rmdp))


So the model reproduces the observations reasonably well on visual inspection. Modelled values of $R_{max}$ are generally less than 50 km for the most intense storms, while for weak storms, $R_{max}$ values tend to be higher, with maximum values occuring for those storms with $\Delta p < 20$ hPa. 

The overall distribution of $R_{max}$ is also well reproduced. Here, we present the distribution from the $R_{max}$ model with the fitted log-normal distribution for the observations. There's a slight over-representation of smaller storms ($R_{max} < 25$ km), but above this the distributions match well.

In [30]:
fig, ax = plt.subplots(1, 1)
ax = sns.kdeplot(lat, rmax, cmap='Reds', kwargs={'levels':levs}, shade=True, shade_lowest=False)
ax = sns.kdeplot(l0, rm, cmap='Blues', kwargs={'levels':levs})
ax.set_xlim(-30, 0)
ax.set_xlabel("Latitude")
ax.set_ylabel(r"$R_{max}$ (nm)")
ax.set_ylim(0, 100)
ax.grid(True)



ax.text(-5, 90, "Observed", color=red)
ax.text(-5, 80, "Model", color=blue)
ax.text(-5, 70, r"$l_2=${0:.3f}".format(l2rmlat))

In [31]:
x = np.arange(1, 101)
v = stats.lognorm.pdf(x, shape, loc=loc, scale=scale)
fig, ax = plt.subplots(1, 1)
sns.distplot(rm, bins=np.arange(0, 101, 5),
             kde_kws={'clip':(0, 100), 'label':"Model data (KDE)"},)
ax.plot(x, v, label="Lognormal fit from observations", color='r')
ax.legend(loc=0)
ax.set_xlabel(r'$R_{max}$ (km)')
ax.set_xlim((0, 100))

In [32]:
fig, ax = plt.subplots(1,1)
sns.distplot(rmax, bins=np.arange(0, 151, 10),
             kde_kws={'clip':(0, 150), 'label':"Observations"}, ax=ax, 
             hist_kws={ "linewidth":3})
sns.distplot(rm, bins=np.arange(0, 151, 10),
             kde_kws={'clip':(0, 150), 'label':"Model"}, ax=ax, color='r',
             hist_kws={ "linewidth":3})
ax.set_ylabel("Probability")
ax.set_xlabel(r"$R_{max}$ (km)")
ax.set_xlim((0, 120))

## Summary

A cursory examination of the AIC scores from these three models indicates the Powell _et al._ (2005) model provides the best balance between magnitude of residuals and degrees of freedom. 

Heuristically, in the limit of high intensity the simplest model will lead to $\lim_{\Delta p\to\infty}R_{max} = 0$, which is physically unrealistic. The Wang and Rosowsky model is comparatively less reliable than the Powell _et al._ model across the observed range. And in fact, the OLS model performs better than the corresponding GLS model, using the same form as Powell _et al._ This latter form will have increasing $R_{max}$ for large $\Delta p$

So our final model (based on southern hemisphere observations) is:

$\ln R_{max} = 4.4726 -0.04057 \Delta p + 0.000313182 \Delta p^2 + 0.0001455 \lambda^2 + \varepsilon$

where $\varepsilon = \mathcal{N}(0, 0.353)$.

<a id='references'></a>
## References

1. Powell, M., Soukup, G., Cocke, S., Gulati, S., Morisseau-Leroy, N., Hamid, S., Dorst, N. and Axe, L. (2005): State of Florida hurricane loss projection model: Atmospheric science component. _Journal of Wind Engineering and Industrial Aerodynamics_, __93__, pp 651-674.
2. Vickery, P. J., Skerlj, P. F. and Twisdale, L. A. (2000): Simulation of Hurricane Risk in the U.S. Using Empirical Track Model. _Journal of Structural Engineering_, __126__, pp 1222-1237.
3. Wang, Y. and Rosowsky, D. V. (2012): Joint distribution model for prediction of hurricane wind speed and size. _Structural Safety_, __35__, pp 40-51.

In [48]:
dparray = np.arange(10, 51, 5)
latarray = np.arange(-23, -5, 2)
testinput = np.column_stack((dparray, latarray))
np.random.seed(10)
print(result.params)
yy = result.eval(x=testinput) #+ np.random.normal(scale=0.335)
print(np.exp(yy))

In [46]:
eps = 0#np.random.normal(0, scale=0.335)
rmw = np.exp(result.eval(x=np.column_stack((np.array([25]), np.array([-15]))))+eps)
print(rmw)

In [47]:
eps = 0#np.random.normal(0, scale=0.335)
newparams = rmod.make_params(alpha=3.5, beta=-0.004, gamma=.7, delta=.002, zeta=.001)
yy = result.eval(params=newparams, x=testinput)+eps
rmw = np.exp(yy)
print(rmw)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(dp, np.abs(lat), rmax, c=rmax, cmap=sns.light_palette('blue', as_cmap=True))
ax.set_xlabel(r"$\Delta p$ (hPa)")
ax.set_ylabel(r"$|\lambda|$ (degrees)")
ax.set_zlabel(r"$R_{max}$ (km)")

In [ ]:
xbins = np.arange(0, 101, 1)
ybins = np.arange(0, 30, 1)
(count, xedge, yedge, img) = plt.hist2d(dp, np.abs(lat), bins=[xbins, ybins], weights=rmax, normed=True)
plt.xlabel(r"$\Delta p$ (hPa)")
plt.ylabel(r"$|\lambda|$ (degrees)")
plt.colorbar(label=r"$R_{max}$")

In [ ]:
from thinkbayes import Pmf, Suite

In [ ]:
#rm = filterPoci(rmax, poci)
print(len(rmax))
pmf = Pmf()
for r in rmax:
    pmf.Incr(r, 1)
    
pmf.Normalize()
for x,y in pmf.Items():
    print( x, y)

In [ ]:
def log_prior(theta):
    alpha, beta, gamma, zeta, sigma = theta
    if sigma < 0:
        return -np.inf  # log(0)
    else:
        return -1.5 * np.log(1 + beta ** 2 + gamma ** 2 + zeta ** 2) - np.log(sigma)

def log_likelihood(theta, x1, x2, x3, y):
    alpha, beta, gamma, zeta, sigma = theta
    y_model = alpha + beta * x1 + gamma * x2 + zeta * x3
    return -0.5 * np.sum(np.log(2 * np.pi * sigma ** 2) + (y - y_model) ** 2 / sigma ** 2)

def log_posterior(theta, x1, x2, x3, y):
    return log_prior(theta) + log_likelihood(theta, x1, x2, x3, y)

In [ ]:
ndim=5
nwalkers=50
nburn=1000
nsteps=2000
np.random.seed(0)
starting_guesses=np.random.random((nwalkers, ndim))


In [ ]:
import emcee

In [ ]:
ydata = rm
xdata1 = dp
xdata2 = dp * dp
xdata3 = np.abs(lat)**2
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_posterior, args=[xdata1, xdata2, xdata3, ydata])

In [ ]:
sampler.run_mcmc(starting_guesses, nsteps)

In [ ]:
emcee_trace = sampler.chain[:, nburn:, :].reshape(-1, ndim).T

In [ ]:
def compute_sigma_level(trace1, trace2, nbins=20):
    """From a set of traces, bin by number of standard deviations"""
    L, xbins, ybins = np.histogram2d(trace1, trace2, nbins)
    L[L == 0] = 1E-16
    logL = np.log(L)

    shape = L.shape
    L = L.ravel()

    # obtain the indices to sort and unsort the flattened array
    i_sort = np.argsort(L)[::-1]
    i_unsort = np.argsort(i_sort)

    L_cumsum = L[i_sort].cumsum()
    L_cumsum /= L_cumsum[-1]
    
    xbins = 0.5 * (xbins[1:] + xbins[:-1])
    ybins = 0.5 * (ybins[1:] + ybins[:-1])

    return xbins, ybins, L_cumsum[i_unsort].reshape(shape)


def plot_MCMC_trace(ax, xdata1, xdata2, xdata3, ydata, trace, scatter=False, **kwargs):
    """Plot traces and contours"""
    xbins, ybins, sigma = compute_sigma_level(trace[0], trace[1])
    ax.contour(xbins, ybins, sigma.T, levels=[0.683, 0.955], **kwargs)
    if scatter:
        ax.plot(trace[0], trace[1], ',k', alpha=0.1)
    ax.set_xlabel(r'$\alpha$')
    ax.set_ylabel(r'$\beta$')
    
    
def plot_MCMC_model(ax, xdata1, xdata2, xdata3, ydata, trace):
    """Plot the linear model and 2sigma contours"""
    ax.plot(xdata1, ydata, 'ok')

    alpha, beta, gamma, zeta = trace[:4]
    xfit = np.linspace(-20, 120, 10)
    yfit = alpha[:, None] + beta[:, None] * xfit
    mu = yfit.mean(0)
    sig = 2 * yfit.std(0)

    ax.plot(xfit, mu, '-k')
    ax.fill_between(xfit, mu - sig, mu + sig, color='lightgray')

    ax.set_xlabel(r'$\Delta p$ (hPa)')
    ax.set_ylabel(r'$R_{max}$ (km)')

def plot_MCMC_results(xdata1, xdata2, xdata3, ydata, trace, colors='k'):
    """Plot both the trace and the model together"""
    fig, ax = plt.subplots(1, 2, figsize=(10, 4))
    plot_MCMC_trace(ax[0], xdata1, xdata2, xdata3, ydata, trace, True, colors=colors)
    plot_MCMC_model(ax[1], xdata1, xdata2, xdata3, ydata, trace)
    fig.tight_layout()

In [ ]:
plot_MCMC_results(xdata1, xdata2, xdata3, ydata, emcee_trace)

In [ ]:
import corner
fig = corner.corner(emcee_trace.T, labels=[r"$\alpha$", r"$\beta$", r"$\gamma$", r"$\zeta$", r"$\ln\,f$"], 
                    plot_density=False, no_fill_contours=True, data_kwargs={'color':'r'})

In [ ]:
import pymc
print(pymc.__version__)

In [ ]:
alpha = pymc.Normal('alpha', 0, 1)

@pymc.stochastic(observed=False)
def beta(value=0):
    return -1.5*np.log(1+value**2)

@pymc.stochastic(observed=False)
def gamma(value=0):
    return -1.5*np.log(1+value**2)

@pymc.stochastic(observed=False)
def zeta(value=0):
    return -1.5*np.log(1+value**2)

@pymc.stochastic(observed=False)
def sigma(value=1):
    return -np.log(abs(value))

@pymc.deterministic
def y_model(x1=xdata1, x2=xdata2, x3=xdata3, alpha=alpha, beta=beta, gamma=gamma, zeta=zeta):
    return alpha + beta * x1 + gamma * x2 + zeta * x3

y = pymc.Normal('y', mu=y_model, tau=1. / sigma ** 2, observed=True, value=ydata)

model1 = dict(alpha=alpha, beta=beta, gamma=gamma, zeta=zeta, sigma=sigma, y_model=y_model, y=y)

In [ ]:
S = pymc.MCMC(model1)
S.sample(iter=10000,burn=5000)

In [ ]:
pymc_trace = [S.trace('alpha')[:],
              S.trace('beta')[:],
              S.trace('gamma')[:],
              S.trace('zeta')[:],
              S.trace('sigma')[:]]
plot_MCMC_results(xdata1, xdata2, xdata3, ydata, pymc_trace)

In [ ]:
fig = corner.corner(np.array(pymc_trace).T, labels=[r"$\alpha$", r"$\beta$", r"$\gamma$", r"$\zeta$", r"$\ln\,f$"], 
                    plot_density=False, no_fill_contours=True, data_kwargs={'color':'r'})

In [ ]:
from scipy.optimize import curve_fit
def func(x, a, b, c, d, f):
    dp = x[:,0]
    lat = x[:,1]
    return a + b*dp + c*np.exp(-d*dp*dp) + f*np.abs(lat)

xx = np.column_stack((dp, lat))

popt, pcov = curve_fit(func, xx, np.log(rmax))
perr = np.sqrt(np.diag(pcov))
print(popt)
print(perr)

In [ ]:
nx = len(dp)
ind = np.random.choice(np.arange(nx), nx, replace=True)
dp0 = dp[ind]
l0 = lat[ind]
xx = np.column_stack((dp0, l0))
yy = func(xx, *popt) + np.random.normal(scale=0.3, size=nx)
rm = np.exp(yy)

fig, ax = plt.subplots(1, 1)
ax.scatter(dp0, rm, c=np.abs(l0), cmap=sns.light_palette('blue', as_cmap=True), s=40, label='Model', alpha=0.5)
ax.scatter(dp, rmax, c='w', edgecolor='r', s=50, marker='+', label='Observations')
ax.set_xlim(0, 100)
ax.set_xlabel(r"$\Delta p$ (hPa)")
ax.set_ylabel(r"$R_{max}$ (km)")
ax.set_ylim(0, 100)
ax.legend(loc=1)
ax.grid(True)

In [ ]:
deltap=np.linspace(0, 200, 200)
lats = np.arange(-30, -1, 4)

fig, ax = plt.subplots(1,1)
sns.set_palette(sns.color_palette("coolwarm", 8))
for l in lats:
    xx = np.column_stack((deltap, l*np.ones(len(deltap))))
    yy = func(xx, *popt)
    ax.plot(deltap, np.exp(yy), label="%d"%l)
    
ax.set_ylabel(r"$R_{max}$ (km)")
ax.set_xlabel(r"$\Delta p$ (hPa)")
ax.legend(loc=1)

In [ ]:
xx = np.column_stack((dp, lat))
yy = func(xx, *popt)
#rm = filterPoci(rmax, poci)

resid = np.log(rmax) - yy
print(yy)
print(np.log(rmax))

In [ ]:
fig, (ax0, ax1) = plt.subplots(1, 2)

ax = sns.distplot(resid, kde_kws={'label':'Residuals', 'linestyle':'--'}, ax=ax0, norm_hist=True)
pp = sm.ProbPlot(resid, stats.norm, fit=True)
pp.qqplot('Normal', 'Residuals', line='45', ax=ax1, color='gray',alpha=0.5)
fig.tight_layout()

#ppfit = pp.fit_params

x = np.linspace(-2, 2, 1000)

ax0.legend(loc=0)

fp = stats.norm.fit(resid)
ax0.plot(x, stats.norm.pdf(x, fp[0], fp[1]), label='Normal', color='r')
print(fp)
print(stats.mstats.normaltest(resid))
ax0.legend()
#p = list(results.params)